In [3]:
import os, shutil

# If folder doesn't exist, create it
os.makedirs("/root/.kaggle", exist_ok=True)

# Now copy the file: path should match where you uploaded it
shutil.copy("/content/kaggle.json", "/root/.kaggle/kaggle.json")
os.chmod("/root/.kaggle/kaggle.json", 0o600)


In [4]:
!pip install kaggle --quiet

import shutil
import os

# Move kaggle.json to the right location
shutil.copy("/content/kaggle.json", "/root/.kaggle/kaggle.json")
os.chmod("/root/.kaggle/kaggle.json", 0o600)

# Download and extract the CXR dataset
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia
!unzip -q chest-xray-pneumonia.zip


Dataset URL: https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia
License(s): other
 99% 2.28G/2.29G [00:17<00:00, 269MB/s]
100% 2.29G/2.29G [00:17<00:00, 144MB/s]


In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    'chest_xray/train', target_size=(224,224), batch_size=32, class_mode='binary')
val_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    'chest_xray/val', target_size=(224,224), batch_size=32, class_mode='binary')
test_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    'chest_xray/test', target_size=(224,224), batch_size=32, class_mode='binary')


Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [6]:
import tensorflow as tf

base = tf.keras.applications.MobileNetV2(input_shape=(224,224,3), include_top=False, weights='imagenet')
base.trainable = False
model = tf.keras.Sequential([
    base,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [7]:
model.fit(train_gen, validation_data=val_gen, epochs=3)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/3
163/163 ━━━━━━━━━━━━━━━━━━━━ 65s 332ms/step - accuracy: 0.8303 - loss: 0.3711 - val_accuracy: 0.7500 - val_loss: 0.4910
Epoch 2/3
163/163 ━━━━━━━━━━━━━━━━━━━━ 49s 301ms/step - accuracy: 0.9611 - loss: 0.1311 - val_accuracy: 0.8750 - val_loss: 0.4205
Epoch 3/3
163/163 ━━━━━━━━━━━━━━━━━━━━ 49s 298ms/step - accuracy: 0.9649 - loss: 0.1060 - val_accuracy: 0.8750 - val_loss: 0.3812


In [8]:
loss, acc = model.evaluate(test_gen)
print(f'Test accuracy: {acc:.2%}')


20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 228ms/step - accuracy: 0.8066 - loss: 0.4454
Test accuracy: 81.73%


In [9]:
model.save('cxr_pneumonia_model.h5')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')